# **Notebook 5a - Visualize Near-Shortest Routes**

---

## **Overview**
This notebook produces illustrative maps for individual origin–destination (OD) pairs, showing how alternative routes overlap or diverge depending on city structure and network characteristics.

These visualizations are essential for interpreting the DiverCity measure, as they help illustrate cases of:
- **Low DiverCity**: strong overlap among near-shortest routes (high route concentration).  
- **High DiverCity**: spatially diverse near-shortest routes with minimal overlap.

---

Theis notebook may help generating:
- **Route Maps** - Showing spatial overlap and route diversity for selected OD pairs.  
- **Map Insets** - Zoomed visualizations for specific OD pairs.  
- **Optional Overlays** - Highlighting mobility attractors or high-traffic corridors.

These visualizations can be used to create figures similar to *Figure 1a–b* and *Figure 4* of the DiverCity paper,  which illustrate examples of low- and high-diversification trips and the spatial distribution of near-shortest routes. The notebook can also be adapted to visualize additional cities, configurations, or speed-reduction scenarios.

In [ ]:
import gzip
import os
import osmnx as ox
from divercity_utils import get_attractors_by_road_types

import lz4.frame
import msgpack

import matplotlib.pyplot as plt
from route_plotting import visualize_alternative_routes
from my_utils import create_folder_if_not_exists

## **1. Load the road networks**

In [ ]:
# ------------------------------------------------------------
# Parameters and Configuration
# ------------------------------------------------------------

# Type of network to visualize ('drive', 'walk', 'bike', etc.)
network_type = "drive"

# Radius of the city area (in meters) — Must match the extraction radius used in Notebook 1
radius_city_m = 10_000

# Identifier for the experiment (used for loading route data and naming outputs)
exp_id = "exp_osm"

# List of cities to visualize (can include multiple entries for comparison)
list_cities = ["milan"]

# Dictionaries to store loaded data
dict_road_network = {}   # Stores the OSM road network for each city
dict_routes = {}         # Stores computed near-shortest routes for each city

## **2. Load the Road Networks**

This section loads the previously downloaded and simplified road networks for the selected cities.  
The processed networks are stored in the `dict_road_network` dictionary for later visualization.

In [ ]:
# ------------------------------------------------------------
# Load and prepare the road networks
# ------------------------------------------------------------

for city in list_cities:
    print(f"\nLoading road network for {city.title()}...")

    # Define paths for compressed and uncompressed files
    network_file = f"../data/road_networks/{city}_{network_type}_{radius_city_m}.graphml.gz"
    uncompressed_network_file = f"../data/road_networks/{city}_{network_type}_{radius_city_m}.graphml"

    # Decompress the network file (OSMnx cannot read compressed .gz directly)
    with gzip.open(network_file, "rb") as f_in:
        with open(uncompressed_network_file, "wb") as f_out:
            f_out.write(f_in.read())

    # Load the decompressed network
    G = ox.load_graphml(uncompressed_network_file)

    # Optionally, remove the uncompressed file after loading
    os.remove(uncompressed_network_file)

    # Extract mobility attractors (motorways and trunks)
    attractors = get_attractors_by_road_types(G, ["motorway", "trunk"])
    G_attractors = G.edge_subgraph(attractors)
    attractor_edges = ox.graph_to_gdfs(G_attractors, nodes=False).reset_index()

    # Extract all road segments (edges)
    edges = ox.graph_to_gdfs(G, nodes=False).reset_index()

    # Build dictionary of geometries for easy access during plotting
    dict_edges_geo = {(u, v): geometry for u, v, geometry in zip(edges["u"], edges["v"], edges["geometry"])}

    # Store results for this city
    dict_road_network[city] = {
        "edges": edges,
        "attractors": attractor_edges,
        "dict_edges_geo": dict_edges_geo
    }

    print(f"  - Loaded {len(edges)} edges ({len(attractor_edges)} attractor edges)")


## **3. Load the Generated Routes**

This section loads the **precomputed routes** generated in *Notebook 2 (Compute DiverCity)*.  
Each file contains the sets of near-shortest routes obtained through Path Penalization for all sampled origin–destination pairs.  

In [ ]:
%%time

for city in list_cities:
    print(f"\nLoading generated routes for {city.title()}...")

    # Define path to the compressed route data
    routes_path = f"../data/results/{city}_{exp_id}/generated_routes.lz4"

    # Read and unpack the serialized data
    with lz4.frame.open(routes_path, "rb") as f:
        packed_data = f.read()
        tmp_dict_routes = msgpack.unpackb(packed_data, strict_map_key=False)

    # Store the unpacked routes in the main dictionary
    dict_routes[city] = tmp_dict_routes

    print(f"  - Loaded!")

## **4. Plot an OD Pair and Its Alternative Routes**

This section visualizes the set of **near-shortest routes** connecting a specific origin–destination (OD) pair within a given radius.  

Each route is color-coded according to its feasibility:  
- **Blue** — Near-Shortest Routes (NSR), i.e., routes within the ε-threshold of the fastest path.  
- **Red** — Non-feasible routes, exceeding the ε tolerance (too long to be considered realistic alternatives).  

Mobility attractors (e.g., highways and trunks) are plotted in **orange** for reference, showing how high-speed infrastructures influence route overlap and diversity.  

This type of visualization helps interpret DiverCity at the trip level, similar to the examples shown in the DiverCity paper (*Figure 1a–b* and *Figure 4*).


In [ ]:
# ------------------------------------------------------------
# Plot alternative routes for a specific OD pair
# ------------------------------------------------------------

city = "milan"

# Trip and visualization parameters
radius = 10              # Radius (km) corresponding to the sampling circle
origin = 0               # Index of the origin node within the circle
destination = 2          # Index of the destination node within the circle

p = 0.1                  # Path penalization factor
eps = 0.3                # Near-shortest route threshold (±30%)
k = 5                    # Maximum number of routes to display

# Create the figure and axis
fig, ax = plt.subplots(figsize=(5, 5))

# Retrieve route alternatives and network data
alternatives = dict_routes[city][radius][f"{origin}_{destination}"][p]
gpd_edges = dict_road_network[city]["edges"]
dict_edges_geo = dict_road_network[city]["dict_edges_geo"]
attractor_edges = dict_road_network[city]["attractors"]

# Visualize the alternative routes
visualize_alternative_routes(
    alternatives,
    gpd_edges,
    dict_edges_geo,
    eps=eps,
    max_k=k,
    ax=ax,
    plot_road_network=True,
    filter_road_types=["residential", "unclassified"]  # Optional filtering for clarity
)

# Overlay mobility attractors (e.g., highways, trunks)
attractor_edges.plot(color="orange", ax=ax, linewidth=4, alpha=0.8)

# Final styling
ax.set_title(f"{city.title()} — Radius {radius} km, OD Pair ({origin}, {destination})", fontsize=11)
ax.set_axis_off()

create_folder_if_not_exists("./Figures/")
save_path = f"./Figures/{city}_routes_r{radius}_o{origin}_d{destination}.pdf"
plt.savefig(save_path, dpi=300, bbox_inches="tight")